In [0]:
#importing required libraries
import numpy as np
from math import factorial
import matplotlib.pyplot as plt

In [0]:
#Initializing the given parameters
Herd=12
gamma = 0.9
epsilon = 0.05

In [0]:
#Caluculating nCr 
n = r = Herd
C=[[0 for i in range(Herd+1)] for j in range(Herd+1)]
for i in range(Herd+1):
    for j in range(i+1):
        if(j==0 or j==i):
            C[i][j]=1
        else:
            C[i][j]=factorial(i)/(factorial(j)*factorial(i-j)) 
  

In [0]:
#Initializing the state and action space sizes
stateSpaceSize = actionSpaceSize = 0

In [0]:
#Initializing the dictionaries of mapping of states and indexes
stateToIndex = dict()
indexToState = dict()

In [0]:
#Rewards and Pay-off initialization
utility = np.array([[0.3,0.4,0.2]])
payoff = np.array([[2.,6,4]])

In [0]:
#Initializating the probabilites
prob_transition = np.array([[0.9,0.1,0.0],[0,0.75,0.25],[0.0,0.15,0.85]])
prob_offspring = np.array([[1.0,0.0,0.0],[0.05,0.8,0.15],[1.0,0.0,0.0]])

In [0]:
#caluculating expected reward
def getReward(state,action):
  return np.sum((state - action)*payoff + (action*utility))

In [0]:
#Method to get all possible states for given herd size
def getStates(H):
  global stateSpaceSize,actionSpaceSize
  count = 0
  for i in range(H+1):
    for j in range(H+1):
      for k in range(H+1):
        if i+j+k <= H:
          stateToIndex[(i,j,k)] = count
          indexToState[count] = (i,j,k)
          count+=1
          stateSpaceSize = actionSpaceSize = count
          
  

In [0]:
getStates(Herd)

In [0]:
#Calculate all possible actions for given state
def getActions(state):
  actions=[]
  for i in range(state[0]+1):
    for j in range(state[1]+1):
      for k in range(state[2]+1):
        actions.append((i,j,k))
  return actions

In [0]:
#Creating table for storing 'v' values
v_Table = np.array([0.0 for i in range(stateSpaceSize)])

In [0]:
#Calculate reward during breeding
def getRewardafterState(state):
  max_child = Herd - (state[0]+state[1]+state[2])
  reward = 0
  for r in range(state[1]+1):
    for q in range(state[1]+1-r):
      p = state[1]-q-r
      n = q+2*r
      if n > max_child:
        break
      astate = (state[0]+n,state[1],state[2])
      ind = stateToIndex[astate]
      reward += v_Table[ind] * pow(prob_offspring[1][0],p) * pow(prob_offspring[1][1],q) * pow(prob_offspring[1][2],r)*C[state[1]][r]*C[state[1]-r][q]
  return reward

In [0]:
#Method to calculate expected reward of next state
def getExpectedReward(state):
  reward = 0
  for p in range(state[0]+1):
    for q in range(state[1]+1):
      for r in range(state[2]+1):
        newState=(state[0]-p,state[1]-q+p+r,state[2]-r+q) 
        reward += C[state[0]][p]*C[state[1]][q]*C[state[2]][r]*getRewardafterState(newState)*pow(prob_transition[0][1],p)*pow(prob_transition[1][2],q)*pow(prob_transition[2][1],r)*pow(1-prob_transition[0][1],state[0]-p)*pow(1-prob_transition[1][2],state[1]-q)*pow(1-prob_transition[2][1],state[2]-r)
  return reward

In [0]:
#Value iteration
n=0
v_Table_new = np.array([0.0 for i in range(stateSpaceSize)])
pi = [(0,0,0) for i in range(stateSpaceSize)] #creating policy table to store optimal policy
min_err = epsilon * (1 - gamma)/(2 * gamma)
while True:
  n+=1
  print("current Iteration :- ",n,",",end="")
  for i in stateToIndex.keys():
    for j in getActions(i):
      e_reward = getReward(np.array(i).reshape(1,3),np.array(j).reshape(1,3)) #E_r/s,a
      c_reward = getExpectedReward((i[0]-j[0],i[1]-j[1],i[2]-j[2]))
      reward = e_reward + gamma * c_reward
      if reward > v_Table_new[stateToIndex[i]]:
        v_Table_new[stateToIndex[i]] = reward
        pi[stateToIndex[i]] = j
  err = 0
  for i in range(v_Table.shape[0]):
    err += ((v_Table[i] - v_Table_new[i])**2)
    v_Table[i] = v_Table_new[i]
    v_Table_new[i] = 0.0
  if n==1:
    v1=np.copy(v_Table)
  if n==10:
    v10=np.copy(v_Table)
  err = np.sqrt(err)
  print("Error is ",err)
  if err < min_err:
    break

current Iteration :-  1 ,Error is  822.8244041106219
current Iteration :-  2 ,Error is  501.05635561010524
current Iteration :-  3 ,Error is  469.9093497945953
current Iteration :-  4 ,Error is  417.90854553477857
current Iteration :-  5 ,Error is  376.4424729904833
current Iteration :-  6 ,Error is  335.4809676033914
current Iteration :-  7 ,Error is  299.3230608414669
current Iteration :-  8 ,Error is  267.1059495941237
current Iteration :-  9 ,Error is  238.82023962851062
current Iteration :-  10 ,Error is  214.01958396784792
current Iteration :-  11 ,Error is  191.99034381218044
current Iteration :-  12 ,Error is  172.29854294535517
current Iteration :-  13 ,Error is  154.44272532052875
current Iteration :-  14 ,Error is  138.36963759659133
current Iteration :-  15 ,Error is  123.93055337018986
current Iteration :-  16 ,Error is  110.97793341645108
current Iteration :-  17 ,Error is  99.3658661475336
current Iteration :-  18 ,Error is  88.95755728032576
current Iteration :-  19 ,Er

In [0]:
indexToState[np.argmax(v_Table)],pi[np.argmax(v_Table)]

In [0]:
np.max(v_Table) 

In [0]:
#Method to convert states into base 13
def getEncryptedStatesBase13():
  return np.array([ 169*i[0] + 13*i[1] + i[2] for i in stateToIndex.keys()])
    

In [0]:
x=getEncryptedStatesBase13()

In [0]:
#plot for States vs V-Table at 1
plt.plot(x,v1)
plt.show()

In [0]:
#plot for States vs V-Table at 10
plt.plot(x,v10)
plt.show()

In [0]:
#plot for States vs optimal V-Table 
plt.plot(x,v_Table)
plt.show()

In [0]:
x[np.argmax(v10)]